In [1]:
%load_ext watermark
%watermark

2019-04-08T17:01:38-05:00

CPython 3.7.3rc1
IPython 7.3.0

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [12, 8]
np.random.seed(42)

# Optimización de hiperparámetros

Hasta ahora hemos visto una manera relativamente sencilla de ver que valores de los hiperparámetros funcionan mejor, mediante las curvas de validación.

Estas curvas son muy útiles para darnos información a los Data Scientists, pero tienen dos problemas:
- Son métodos gráficos, esto significa que necesitan un humano para interpretarlas y no nos permiten automatizar el proceso para encontrar los hiperparámetros óptimos.
- Solo toman un hiperparámetro a la vez. Esto significa que hacen que sea más dificil el evaluar combinaciones de los hiperparámetros (si quisieramos evaluar multiples hiperparámetros tendriamos que hacer gráficas de planos o hiperplanos).

Vamos a ver ahora métodos más robustos para dado un modelo, encontrar el conjunto de hiperparámetros que hace que funcione mejor.

# Cargamos los datos

Vamos a usar un dataset nuevo, el [Census Income Dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income). Es un dataset que tiene datos demográficos sobre 50,000 personas en Estados Unidos y como variable objetivo tiene una variable booleana (Verdadero/Falso) sobre si dicha persona gana más de 50K$ al año o no.

In [3]:
censo = pd.read_csv("D:/datasets/Curso_Mauel_Garrido/salario_censo.csv")

In [4]:
censo.shape

(32561, 13)

In [5]:
censo.head()

,edad,clase_laboral,nivel_educativo,status_matrimonial,ocupacion,relacion,raza,genero,ganancias_capital,perdidas_capital,horas_laborables,pais_origen,objetivo
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Pipeline de procesamiento de datos

In [6]:
variable_dependiente = "objetivo"
variables_independientes = censo.drop(variable_dependiente, axis=1).columns
censo_X = censo[variables_independientes]
censo_y = censo[variable_dependiente]

In [7]:
censo_y.unique()

array([' <=50K', ' >50K'], dtype=object)

En este caso la variable objetivo está definida como texto, asi que la convertimos a una variable binaria numérica.

In [8]:
censo_y = censo_y.replace({" <=50K": 0, " >50K": 1})

Separamos datos en numéricos y no numéricos. Viendo el [diccionario de datos](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names) del dataset vemos que no hay variables categóricas, solo la variable educacion que ya viene codificada como numérica (*education-num*).

In [9]:
datos_numericos = censo_X.select_dtypes(np.number)
col_numericas = datos_numericos.columns

datos_categoricos = censo_X.select_dtypes([object])
col_no_numericas = datos_categoricos.columns

Vamos a usar un transformador nuevo `MultiLabelBinarizer`. Es como el LabelBinarizer pero funciona para multiples columnas

In [10]:
from sklearn.preprocessing import Imputer, StandardScaler, MultiLabelBinarizer

In [11]:
b = MultiLabelBinarizer()
b.fit_transform(
 [
     ["gato", "patata", "rojo"],
     ["perro", "zanahoria", "azul"],
     ["camello", "patata", "verde"],
     ["gato", "patata", "rojo"]
 ]
)

array([[0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 0]])

Con `classes_` podemos ver los diferentes valores de las variables categoricas

In [12]:
b.classes_

array(['azul', 'camello', 'gato', 'patata', 'perro', 'rojo', 'verde',
       'zanahoria'], dtype=object)

Creamos el transformador `BinarizadorMultipleCategorico` que es básicamente el MultiLabelBinarizer pero "arreglado" para que funcione en Pipelines (Estoy usando sklearn 0.19.0, este bug se arreglará en el futuro).

In [13]:
from sklearn.base import TransformerMixin
from sklearn import preprocessing

class ColumnExtractor(TransformerMixin):
    """Transformador que selecciona columnas de un dataframe"""
    def __init__(self, columns):
        self.columns = columns
        
    def transform(self, X, **transform_params):
        return X[self.columns].as_matrix()
        
    def fit(self, X, y = None, **fit_params):
        return self
    
class BinarizadorMultipleCategorico(preprocessing.MultiLabelBinarizer):
    def fit(self, X, y = None):
        super(BinarizadorMultipleCategorico, self).fit(X)
        
    def transform(self, X, y= None):
        return super(BinarizadorMultipleCategorico, self).transform(X)

    def fit_transform(self, X, y = None):
        return super(BinarizadorMultipleCategorico, self).fit(X).transform(X)

In [14]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [15]:
pipeline_numerico = Pipeline([
    ('selector_numerico', ColumnExtractor(columns = col_numericas)),
    ('imputador', Imputer()),
    ('escalador', StandardScaler()),
])

pipeline_categorico = Pipeline([
    ('selector_categorico', ColumnExtractor(columns = col_no_numericas)),
    ('codificador_numerico', BinarizadorMultipleCategorico()),
])

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
pipeline_categorico.fit_transform(censo_X).shape

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


(32561, 84)

In [17]:
col_numericas

Index(['edad', 'nivel_educativo', 'ganancias_capital', 'perdidas_capital',
       'horas_laborables'],
      dtype='object')

In [18]:
pipeline_numerico.fit_transform(censo_X).shape

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


(32561, 5)

In [19]:
pipeline_procesado = FeatureUnion([
    ('transformacion_numericas', pipeline_numerico),
    ('transformacion_categorica', pipeline_categorico),
])

In [20]:
pipeline_procesado

FeatureUnion(n_jobs=None,
       transformer_list=[('transformacion_numericas', Pipeline(memory=None,
     steps=[('selector_numerico', <__main__.ColumnExtractor object at 0x000002C3BC5BD1D0>), ('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('escalador', StandardScaler(copy=True, with_m...98>), ('codificador_numerico', BinarizadorMultipleCategorico(classes=None, sparse_output=False))]))],
       transformer_weights=None)

In [21]:
censo_X_procesado = pipeline_procesado.fit_transform(censo_X)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


In [22]:
censo_X_procesado.shape

(32561, 89)

Antes que nada vamos a ver que puntuaciones tienen unos cuantos modelos con sus  hiperparámetro por defecto

In [25]:
resultados = {}

In [26]:
from sklearn.model_selection import cross_validate

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="roc_auc", n_jobs=-1, cv=5, return_train_score=True)
    return resultados_estimador

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].max()
    return resultados_df

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [28]:
resultados["reg_logistica"] = evaluar_modelo(LogisticRegression(), censo_X_procesado, censo_y)
resultados["naive_bayes"] = evaluar_modelo(GaussianNB(), censo_X_procesado, censo_y)
resultados["rf"] = evaluar_modelo(RandomForestClassifier(), censo_X_procesado, censo_y)
resultados["svc"] = evaluar_modelo(SVC(), censo_X_procesado, censo_y)

In [29]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logistica,0.843554,0.018744,0.906715,0.908326,0.010787,0.001703,0.997895,0.912449
naive_bayes,0.218701,0.043737,0.781202,0.783341,0.002797,0.003975,0.859760,0.786896
rf,0.731085,0.028119,0.874262,0.995482,0.009349,0.002555,0.962178,1.000000
svc,78.197806,11.003671,0.908628,0.910540,1.000000,1.000000,1.000000,0.914673


Vamos a seleccionar un estimador en función de los resultados iniciales y optimizarlo. Elijo el estimador Random Forest por que funciona muy bien en comparación a los demás y es bastánte rápido de entrenar.

In [30]:
estimador_rf = RandomForestClassifier()

Scikit-learn tiene dos métodos de optimización de hiperparámetros, [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) y [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV).

`GridSearchCV` funciona realizando una busqueda en una malla, es decir, pasandole un conjunto de posibles opciones de hiperparámetros evalua de forma completa cada combinación de dichos parámetros (es decir, el valor 1 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, el valor 2 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, etcétera).

La ventaja de utilizar una búsqueda de malla es que nos aseguramos de que se han probado todas las combinaciones posibles. El problema es que el proceso requiere mucho tiempo de computación, y según que dataset usemos 

In [33]:
%%timeit
import time
def foo():
    time.sleep(1)

266 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%%timeit -n 1 -r 1  #n 1 dice que ejecute esta celda solo una vez, -r 1 que ejecute un solo loop
def foo():
    time.sleep(1)

400 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [36]:
print(estimador_rf.__doc__)

A random forest classifier.

    A random forest is a meta estimator that fits a number of decision tree
    classifiers on various sub-samples of the dataset and uses averaging to
    improve the predictive accuracy and control over-fitting.
    The sub-sample size is always the same as the original
    input sample size but the samples are drawn with replacement if
    `bootstrap=True` (default).

    Read more in the :ref:`User Guide <forest>`.

    Parameters
    ----------
    n_estimators : integer, optional (default=10)
        The number of trees in the forest.

        .. versionchanged:: 0.20
           The default value of ``n_estimators`` will change from 10 in
           version 0.20 to 100 in version 0.22.

    criterion : string, optional (default="gini")
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "entropy" for the information gain.
        Note: this parameter is tree-specific.

    max_depth :

In [37]:
estimador_rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Vamos a definir los límites de la búsqueda de hiperparámetros.

In [38]:
np.linspace(10,1000,10).astype(int)

array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])

In [40]:
parametros_busqueda_rf = {
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
    "class_weight": [None, "balanced"]
}

In [41]:
grid = GridSearchCV(estimator = estimador_rf, 
                    param_grid = parametros_busqueda_rf,
                    scoring = "roc_auc", n_jobs = -1)

`GridSearchCV` se comporta como un estimador en cuanto a que tiene un metodo fit que usamos para "entrenarlo" y que realize la búsqueda en malla.

Para ver cuanto tiempo tarda en realizar la búsqueda usamos la mágia de Jupyter notebook `%%timeit` que evalua el tiempo que tarda una función en ejecutarse


In [42]:
%%timeit -n 1 -r 1
grid.fit(censo_X_procesado, censo_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


17min 54s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi ordenador la busqueda en malla ha tardado 7minutos y 49 segundos 

Ahora podemos ver la puntuación que ha obtenido el mejor estimador así como los parámetros del mismo

In [43]:
print(grid.best_score_)
print(grid.best_estimator_)

0.8973722564825424
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=780, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Tras haberlo ajustado, Gridsearch nos devuelve el ranking de todas las variantes evaluadas junto con métricas de su funcionamiento con el atributo `cv_results_`

In [45]:
pd.DataFrame(grid.cv_results_).sort_values(by="rank_test_score")

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
17,46.176672,0.743398,3.342973,2.551143e-02,None,entropy,780,"{'class_weight': None, 'criterion': 'entropy',...",0.894313,0.896849,0.900955,0.897372,0.002737,1,0.998242,0.998151,0.998040,0.998144,0.000083
16,38.157726,1.052273,3.103431,3.437397e-01,None,entropy,670,"{'class_weight': None, 'criterion': 'entropy',...",0.894590,0.896329,0.900542,0.897154,0.002499,2,0.998217,0.998133,0.998036,0.998128,0.000074
19,56.528426,0.174090,4.426032,4.101676e-02,None,entropy,1000,"{'class_weight': None, 'criterion': 'entropy',...",0.894219,0.896328,0.900782,0.897109,0.002736,3,0.998207,0.998142,0.998037,0.998129,0.000070
18,50.878694,0.121674,3.780363,3.374661e-02,None,entropy,890,"{'class_weight': None, 'criterion': 'entropy',...",0.894154,0.896449,0.900382,0.896995,0.002572,4,0.998219,0.998145,0.998053,0.998139,0.000068
15,31.305159,0.091977,2.317168,1.947881e-02,None,entropy,560,"{'class_weight': None, 'criterion': 'entropy',...",0.894229,0.896100,0.900372,0.896900,0.002571,5,0.998212,0.998136,0.998026,0.998125,0.000076
7,42.073466,0.162007,3.369003,5.154582e-02,None,gini,780,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893878,0.896152,0.900261,0.896764,0.002641,6,0.998223,0.998153,0.998034,0.998136,0.000078
14,25.197209,0.045978,1.843321,1.275893e-02,None,entropy,450,"{'class_weight': None, 'criterion': 'entropy',...",0.893480,0.896597,0.900106,0.896728,0.002707,7,0.998230,0.998146,0.998023,0.998133,0.000085
6,36.454998,0.250044,2.889946,3.375194e-02,None,gini,670,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893930,0.895824,0.900354,0.896703,0.002695,8,0.998234,0.998134,0.998039,0.998136,0.000079
8,48.561528,0.269463,3.874090,2.209057e-02,None,gini,890,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893569,0.896251,0.900232,0.896684,0.002737,9,0.998225,0.998151,0.998028,0.998134,0.000081
9,54.768409,0.413499,4.353146,9.573045e-02,None,gini,1000,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893440,0.896140,0.900286,0.896622,0.002815,10,0.998219,0.998152,0.998041,0.998138,0.000073


`GridSearchCV` al estar ajustado se convierte en un estimador, por lo cual podemos usar el método predict, por debajo simplemente se usará el mejor estimador `grid.best_estimator_`. 

Para añadir el funcionamiento del mejor estimador obtenido por el modelo con nuestra funcion `evaluar_modelo` no usamos el objeto grid en si, ya que la funcion `cross_validate` hace multiples ajustes y evaluaciones (volveriamos a esperar los 8 minutos que a tardado un ajuste multiplicado por el número de validaciones cruzadas!).

Para evaluar el funcionamiento del mejor estimador simplemente usamos la funcion con el mejor estimador directamente.

In [46]:
resultados["rf_gridsearch"] = evaluar_modelo(grid.best_estimator_, censo_X_procesado, censo_y)

Ahora vamos a realizar la misma optimización de parámetros pero usando `RandomizedSearchCV`. RandomizedSearchCV funciona de forma similar a GridSearchCV, pero en vez de evaluar todas las combinaciones posibles de hiperparámetros, se toman n muestras de hiperparámetros de dichas distribuciones.

Se recomienda usar distribuciones en vez de valores fijos para hiperparámetros continuos.

Primero vamos a evaluar el funcionamiento de la busqueda aleatoria con los mísmos hiperparámetros que hemos usado en la busqueda en malla. Para `RandomizedSearchCV` tenemos que indicarle cuantas variantes de hiperparámetros utilizar (definidas por el parámetro n_iter, por defecto toma 10 variantes). Dado que dicha búsqueda toma muestreos el parámetro ya no se llama `param_grid` sino `param_distributions`.

In [48]:
busqueda_random = RandomizedSearchCV(estimator = estimador_rf, 
                    param_distributions = parametros_busqueda_rf,
                   scoring = "roc_auc", n_jobs = -1, n_iter = 10)

In [49]:
%%timeit -n 1 -r 1
busqueda_random.fit(censo_X_procesado, censo_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


3min 56s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


La búsqueda con 10 iteraciones ha tardado 1min 25s en mi máquina. Veamos como ha funcionado.

In [50]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.896911363494159
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=340, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


La búsqueda de malla obtuvo un ROC AUC máximo de 0.89726431782 versus 0.896926092788 obtenido por la búsqueda aleatoria. Sin embargo la busqueda aleatoria ha tardado 8 veces menos!

In [51]:
resultados["rf_randomizedsearch"] = evaluar_modelo(grid.best_estimator_, censo_X_procesado, censo_y)

Una ventaja del Randomized Search es que nos permite evaluar un espacio de hiperparámetros más amplio para un tiempo de computación similar.

Para ver esto vamos a ampliar el espacio de búsqueda de hiperparámetros y hacer 100 muestreos.

In [52]:
from scipy.stats import randint as sp_randint

param_dist_random = {
    "max_depth": [3, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
}

In [53]:
busqueda_random_100 = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=10)

In [54]:
%%timeit -n 1 -r 1
busqueda_random_100.fit(censo_X_procesado, censo_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


2min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi máquina esta búsqueda ha tardado 8 minutos 54 segundos, un poco más que el grid search

In [55]:
print(busqueda_random_100.best_score_)
print(busqueda_random_100.best_estimator_)

0.918683396387896
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=9,
            min_weight_fraction_leaf=0.0, n_estimators=230, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


La búsqueda aleatoria con los nuevos parámetros ha tardado un tiempo similar a la busqueda en malla, pero ha obtenido una puntuación máxima ROC AUC de 0.91950285418 (versus 0.89726431782 de la busqueda en malla)

In [56]:
resultados["rf_randomizedsearch_100"] = evaluar_modelo(busqueda_random_100.best_estimator_,
                                                      censo_X_procesado, censo_y)

Vemos que el estimador obtenido con la búsqueda aleatoria es el que mejor funciona.

En general, salvo que el espacio de hiperparámetros que queramos explorar sea pequeño, es mejor el utilizar `RandomizedSearchCV` en vez de `GridSearchCV`. Esto es así por que en general no existe un unico conjunto de hiperparámetros que obtiene el mejor funcionamiento, sino que suelen existir multiples "areas" en el espacio dimensional de los hiperparámetros que funcionan de forma similar. Al hacer una búsqueda aleatoria podemos explorar las diversas areas en un tiempo más reducido.

# Optimización de parámetros dentro de un Pipeline

Los algoritmos de busqueda de sklearn siguen la API de transformadores y estimadores. Esto significa que podemos crear un pipeline e incluir la optimización de hiperparámetros dentro.

In [57]:
busqueda_random_10 = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=10)

pipeline_estimador = Pipeline(
    [
     ("procesado", pipeline_procesado),
     ("estimador", busqueda_random_10)   
    ])

Ahora podemos ajustar directamente en los datos originales sin tener que preprocesarlos.

In [58]:
pipeline_estimador.fit(censo_X, censo_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Pipeline(memory=None,
     steps=[('procesado', FeatureUnion(n_jobs=None,
       transformer_list=[('transformacion_numericas', Pipeline(memory=None,
     steps=[('selector_numerico', <__main__.ColumnExtractor object at 0x000002C3BC5BD1D0>), ('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbo... random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0))])

In [59]:
pipeline_estimador.predict(censo_X)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

Hay varias librerias externas que permiten hacer optimización de parámetros de forma más flexible y/o compleja que las implementaciones que soporta scikit-learn por defecto.


# Scikit-optimize [(link)](https://scikit-optimize.github.io/)

Scikit-Optimize, o skopt, es una librería que implementa multiples metodos para optimizar hiperparámetros de forma secuencial.

Se instala con:

`{sys.executable} -m pip install scikit-optimize` (existe una version en conda-forge pero solo de una version antigua)

La api de scikit-optimize no es tan similar a la de sklearn como seria posible, no obstante es muy facil de usar. Permite usar diversos algoritmos para ayudar al proceso de optimización, por ejemplo procesos gausianos o Bosques aleatorios

In [86]:
import skopt

In [87]:
skopt.__version__

'0.5.2'

In [88]:
from skopt import gp_minimize 

En vez de usar un diccionario con el espacio de hiperparámetros que queremos buscar, scikit-optimize necesita pasarle una lista de parámetros.

skopt es una libreria relativamente nueva, y tiene ciertas limitaciones comparada con scikitlearn. Por ejemplo, en vez de diccionarios con los nombres de los parámetros,  espera como inputs listas, no se pueden usar funciones de distribuciones (como scipy.randint).

In [89]:
from skopt import space

param_espacio_skopt = [
    space.Integer(3, 10), #max_depth
    space.Integer(1, 11), #max_features
    (0.001, 0.99, "uniform"), #min_samples_split
    (0.001, 0.5, "uniform"), #min_samples_leaf
    space.Integer(1, 1000), #n_estimators
    space.Categorical(["gini", "entropy"]), #criterion,
    space.Categorical([True, False]), #bootstrap
]

Scikit-optimize necesita que definamos la funcion objetivo, que ira variando en funcion de los parámetros elegidos. Dicha función tiene que crear el estimador y evaluarlo y devolver la evaluación.

In [90]:
from sklearn.model_selection import cross_val_score

estimador_rf = RandomForestClassifier()

def funcion_optimizable(params):
    #params es simplemente una selección especifica de hiperparámetros
    max_depth, max_features, min_samples_split, min_samples_leaf,  n_estimators, criterion, bootstrap = params

    estimador_rf.set_params(
                   max_depth=max_depth,
                   max_features=max_features,
                   min_samples_split=min_samples_split, 
                   min_samples_leaf=min_samples_leaf,
                   n_estimators=n_estimators,
                   criterion=criterion
                  )

    return -np.mean(cross_val_score(estimador_rf, censo_X_procesado, censo_y, cv=5, n_jobs=-1,
                                    scoring="roc_auc"))

Ahora podemos dejar que skopt optimize los outputs de la funcion `funcion_optimizable` mediante el uso de `gp_minimize`

In [91]:
%%timeit -n 1 -r 1
resultado_gp = gp_minimize(funcion_optimizable, param_espacio_skopt, n_calls=10, random_state=42)

1min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


este proceso ha tardado 25 minutos en mi maquina con n_call = 100

El parámetro `x` nos da el vector con los parámetros con mejor funcionamiento

In [92]:
resultado_gp.x

NameError: name 'resultado_gp' is not defined

In [93]:
estimador_skopt_gp_100 = RandomForestClassifier(
    max_depth=resultado_gp.x[0],
    max_features=resultado_gp.x[1],
    min_samples_split=resultado_gp.x[2], 
    min_samples_leaf=resultado_gp.x[3],
    n_estimators=resultado_gp.x[4],
    criterion=resultado_gp.x[5]
)

NameError: name 'resultado_gp' is not defined

In [94]:
resultados["rf_skopt_gp_100"] = evaluar_modelo(estimador_skopt_gp_100, censo_X_procesado, censo_y)

NameError: name 'estimador_skopt_gp_100' is not defined

skopt (version >0.4) tambien tiene una implementacion de `BayesSearchCV`, que es un reemplazo de GridSearchCV pero que usa optimización bayesiana (en vez de probar todas las posibilidades).

In [95]:
from skopt import BayesSearchCV 

param_espacio_skopt_bayesCV = {
     "max_depth": space.Integer(3, 10), #
    "max_features": space.Integer(1, 11), #
    "min_samples_split": space.Real(0.001, 0.99, "uniform"), #
    "min_samples_leaf": space.Real(0.001, 0.5, "uniform"), #
    "n_estimators": space.Integer(1, 1000), #
    "criterion": space.Categorical(["gini", "entropy"]),
    "boostrap": space.Categorical([True, False])
}

busqueda_bayesiano_skopt_100 = BayesSearchCV(
    estimator=estimador_rf, 
    search_spaces=param_espacio_skopt_bayesCV,
    scoring="roc_auc", n_jobs=-1, n_iter=100,
    random_state=42
)

In [96]:
%%timeit -n 1 -r 1
busqueda_bayesiano_skopt_100.fit(censo_X_procesado, censo_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: Invalid parameter boostrap for estimator RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.14884047934633715,
            min_samples_split=0.2739631910202954,
            min_weight_fraction_leaf=0.0, n_estimators=166, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [97]:
busqueda_bayesiano_skopt_100.best_estimator_.get_params()

AttributeError: 'BayesSearchCV' object has no attribute 'best_estimator_'

In [98]:
resultados["rf_bayesiano_skopt_100"] = evaluar_modelo(busqueda_bayesiano_skopt_100.best_estimator_,
                                                      censo_X_procesado, censo_y)

In [135]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.066678,0.014363,0.781202,0.783341,0.001043,0.010738,0.849714,0.786865
reg_logistica,0.460645,0.018298,0.906715,0.908326,0.007205,0.013680,0.986234,0.912413
rf,0.395003,0.017742,0.874312,0.995521,0.006179,0.013264,0.950989,1.000000
rf_bayesiano_skopt_100,7.136472,0.193855,0.910383,0.914396,0.111627,0.144932,0.990223,0.918510
rf_gridsearch,3.986320,0.236890,0.848783,0.849026,0.062353,0.177107,0.923221,0.852846
rf_randomizedsearch,4.077757,0.253586,0.854896,0.855122,0.063783,0.189589,0.929870,0.858970
rf_randomizedsearch_100,63.931399,1.337555,0.919371,0.958161,1.000000,1.000000,1.000000,0.962472
rf_skopt_gp_100,1.557633,0.057794,0.906729,0.910607,0.024364,0.043208,0.986249,0.914704


### Hyperopt-sklearn [(link)](https://github.com/hyperopt/hyperopt-sklearn)

Hyperopt-sklearn es una implementación de Hyperopt, que es la librería más famosa para optimización de hiperparámetros) pero preparado para funcionar con scikit-learn

In [102]:
import sys
!{sys.executable} -m pip install hpsklearn

  Stored in directory: C:\Users\jorge\AppData\Local\pip\Cache\wheels\41\ee\c4\3c267cbf78f0905434ee36b915d97a20610ad3af7ff3c75852
  Stored in directory: C:\Users\jorge\AppData\Local\pip\Cache\wheels\0c\61\d2\d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built hpsklearn future


In [107]:
from sklearn.metrics import roc_auc_score
from hpsklearn import HyperoptEstimator, random_forest

optimizador_hpsklearn = HyperoptEstimator( classifier=random_forest('estimador_rf'),
                                       seed=42, loss_fn=roc_auc_score, max_evals=10, trial_timeout=55)

In [108]:
%%timeit -n 1 -r 1
optimizador_hpsklearn.fit(censo_X_procesado, censo_y)

100%|█████████████████████████████████████████████████████| 1/1 [00:57<00:00, 57.03s/it, best loss: 0.6171504172603297]
5min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


`HyperoptEstimator` nos devuelve un estimador que podemos usar como cualquiera de los de sklearn

In [109]:
optimizador_hpsklearn.predict(censo_X_procesado)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

con `best_model()` podemos ver el mejor modelo producido

In [111]:
#estimador_hpsklearn.best_model()
optimizador_hpsklearn.best_model()

{'learner': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=2, max_features=0.32993202225925455,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=75, n_jobs=1, oob_score=False, random_state=2,
             verbose=False, warm_start=False),
 'preprocs': (Normalizer(copy=True, norm='l1'),),
 'ex_preprocs': ()}

Vemos que el estimador de hyperopt-sklearn tambien puede añadir pasos de preprocesamiento automáticos

In [113]:
#modelo_hpsklearn = estimador_hpsklearn.best_model()["learner"]
modelo_hpsklearn = optimizador_hpsklearn.best_model()["learner"]

In [114]:
resultados["rf_hpsklearn_10"] = evaluar_modelo(modelo_hpsklearn, censo_X_procesado, censo_y)

In [115]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logistica,0.843554,0.018744,0.906715,0.908326,0.010787,0.001703,0.987005,0.910349
naive_bayes,0.218701,0.043737,0.781202,0.783341,0.002797,0.003975,0.850378,0.785085
rf,0.731085,0.028119,0.874262,0.995482,0.009349,0.002555,0.951679,0.997699
svc,78.197806,11.003671,0.908628,0.910540,1.000000,1.000000,0.989088,0.912568
rf_gridsearch,50.231964,1.715222,0.896489,0.997770,0.642371,0.155877,0.975873,0.999992
rf_randomizedsearch,51.531673,1.940171,0.896380,0.997778,0.658991,0.176320,0.975755,1.000000
rf_randomizedsearch_100,18.714364,0.396787,0.918653,0.962459,0.239321,0.036059,1.000000,0.964602
rf_hpsklearn_10,2.758735,0.078110,0.885977,0.886425,0.035279,0.007099,0.964430,0.888399


A modo de prueba, he dejado el Optimizador de hyperopt-sklearn toda la noche corriendo. Lo bueno de este estimador es que en cualquier momento podemos parar el proceso (dandole en el notebook a `kernel->interrup` y el optimizador seguira conservando el modelo mejor.

In [118]:
from hpsklearn import any_classifier, any_preprocessing

estimador_final = HyperoptEstimator( classifier=any_classifier("clf"), 
                                    preprocessing=any_preprocessing("preproc"),
                                    seed=42, loss_fn=roc_auc_score, max_evals=20) #max_evals=2000

In [ ]:
estimador_final.fit(censo_X_procesado, censo_y)

In [90]:
estimador_final.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.016925014519856875, loss='deviance',
               max_depth=2, max_features=0.3243309842493083,
               max_leaf_nodes=None, min_impurity_decrease=0.0,
               min_impurity_split=None, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=46, presort='auto', random_state=4,
               subsample=1.0, verbose=0, warm_start=False),
 'preprocs': (PCA(copy=True, iterated_power='auto', n_components=88, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False),)}

In [96]:
modelo_hpsklearn_final = Pipeline([
    ("preprocs", estimador_final.best_model()["preprocs"][0]),
    ("learner", estimador_final.best_model()["learner"])
])

In [97]:
resultados["rf_hpsklearn_final"] = evaluar_modelo(modelo_hpsklearn_final, censo_X_procesado, censo_y)

In [99]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.066678,0.014363,0.781202,0.783341,0.001043,0.010738,0.849714,0.786865
reg_logistica,0.460645,0.018298,0.906715,0.908326,0.007205,0.013680,0.986234,0.912413
rf,0.395003,0.017742,0.874312,0.995521,0.006179,0.013264,0.950989,1.000000
rf_bayesiano_skopt_100,7.136472,0.193855,0.910383,0.914396,0.111627,0.144932,0.990223,0.918510
rf_gridsearch,3.986320,0.236890,0.848783,0.849026,0.062353,0.177107,0.923221,0.852846
rf_hpsklearn_10,11.909622,0.755976,0.889492,0.890850,0.186288,0.565193,0.967500,0.894859
rf_hpsklearn_final,4.160803,0.031582,0.877192,0.880618,0.065082,0.023612,0.954122,0.884580
rf_randomizedsearch,4.077757,0.253586,0.854896,0.855122,0.063783,0.189589,0.929870,0.858970
rf_randomizedsearch_100,63.931399,1.337555,0.919371,0.958161,1.000000,1.000000,1.000000,0.962472
rf_skopt_gp_100,1.329353,0.042163,0.905333,0.909874,0.020793,0.031522,0.984731,0.913968
